In [3]:
import os
from dotenv import load_dotenv

import asyncio

from typing import Dict

from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent

import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

import resend


In [4]:
load_dotenv(override=True)  # Loads the .env file

True

## Step 1: Agent workflow

In [5]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [6]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4.1-nano"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4.1-nano"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4.1-nano"
)

In [ ]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Simplify Your SOC2 Compliance & Prepare for Audits Effortlessly

Dear [Recipient’s Name],

I hope this message finds you well. As organizations like yours strive to maintain robust security standards, ensuring smooth SOC2 compliance and audit readiness can often be a complex and time-consuming process.

At ComplAI, we leverage advanced AI technology to streamline your compliance efforts, providing real-time insights, automated documentation, and continuous monitoring. Our platform helps you identify gaps proactively, reduce manual effort, and stay audit-ready at all times.

Would you be open to a brief call next week to discuss how ComplAI can simplify your SOC2 compliance journey?

Looking forward to your response.

Best regards,  
[Your Name]  
[Your Title]  
ComplAI  
[Your Contact Information]  
[Website URL]

In [13]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
    Imagine you are a customer and pick the one you are most likely to respond to. \
    give very very simple explanation why you choose; reply with the selected email .",
    model="gpt-4.1-nano"
)

In [14]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    for output in outputs:
        print(output + "\n\n")

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")

Subject: Simplify Your SOC2 Compliance and Audits with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. Achieving and maintaining SOC2 compliance can be a complex and time-consuming process, but it doesn't have to be that way.

At ComplAI, we offer an AI-powered SaaS platform designed to streamline your compliance efforts and prepare you for audits with ease. Our solution provides real-time assessments, automated documentation, and proactive risk management—so you can focus on what matters most.

Would you be open to a brief call to see how ComplAI can help simplify your SOC2 journey?

Looking forward to your thoughts.

Best regards,  
[Your Name]  
[Your Position]  
ComplAI  
[Your Contact Information]


Subject: Is Your SOC2 Audit as Smooth as a Buttered Slide? 🧈🚀

Hi [Name],

Let’s face it—SOC2 audits can sometimes feel like trying to juggle flaming torches blindfolded.🔥🎩 But what if I told you there’s a way to make that process so seamless, you could do it weari

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [15]:
sales_agent1

Agent(name='Professional Sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoffs=[], model='gpt-4.1-nano', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [7]:
@function_tool
def send_email(body: str):
    """Send out an email with the given body using Resend only."""
    params: resend.Emails.SendParams = {
        "from": "Acme <onboarding@resend.dev>",
        "to": [os.environ["MY_GMAIL_ADDRESS"]],
        "subject": "Sales email from ComplAI",
        "html": f"<div>{body}</div>",
    }

    email = resend.Emails.send(params)
    return {"status": "success", "id": getattr(email, "id", None), "raw": email}


In [8]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body using Resend only.', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001CB07599800>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [9]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)



In [10]:
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001CB0759A020>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [11]:
tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001CB0759A020>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001CB0759A340>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3', descr

## And now it's time for our Sales Manager - our planning agent

In [17]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.


You are a Sales Agent. Create a cold sales email.
Rules:
- Output must be valid HTML.
- Use <p> for paragraphs.
- Use <strong> instead of **bold**, and <em> instead of *italics*.
- No markdown symbols like *, **, or ##.

"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-5-nano")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)